Connected to Python 3.10.11

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import OneHotEncoder

# chargement de données
file_path = 'C:/Users/YOUSRA/Desktop/SOA Research;EarthenDam/srcsc-2025-dam-data-for-students.xlsx'
data = pd.read_excel(file_path)
print(data.head(), "\n")

# Étape 1 : Filtrer les données pour inclure uniquement les "earthen dams"
earthen_dams = data[data['Primary Type'] == 'Earth']
print(earthen_dams.head(), "\n")

# Étape 2 : Créer une variable binaire pour la probabilité d'échec sur 10 ans
threshold = 0.5  # 50% probabilité d'échec
earthen_dams['Failure_Binary'] = (earthen_dams['Probability of Failure'] >= threshold).astype(int)
print(earthen_dams[['Probability of Failure', 'Failure_Binary']].head(), "\n")

# Étape 3 : Sélectionner les variables explicatives pertinentes
explanatory_vars = [ 'Region', 'Regulated Dam', 'Primary Purpose','Height (m)', 'Length (km)', 'Volume (m3)', 'Year Completed','Years Modified', 'Surface (km2)', 'Drainage (km2)', 'Spillway', 'Last Inspection Date', 'Inspection Frequency','Distance to Nearest City (km)', 'Hazard', 'Assessment', 'Assessment Date']  
target_var = 'Failure_Binary'

# Isoler les données pertinentes
X = earthen_dams[explanatory_vars]
y = earthen_dams[target_var]
print(X.head(), "\n")
print(y.head(), "\n")


#Les valeurs manquaantessss ???????????

# Étape 4 : Encoder les variables catégoriques
categorical_vars = X.select_dtypes(include=['object']).columns
encoder = OneHotEncoder(sparse_output=False, drop='first')
X_encoded = pd.DataFrame(encoder.fit_transform(X[categorical_vars]), columns=encoder.get_feature_names_out())
X_numeric = X.select_dtypes(exclude=['object'])
X_final = pd.concat([X_numeric.reset_index(drop=True), X_encoded.reset_index(drop=True)], axis=1)
print(X_final.head(), "\n")
# Étape 5 : Diviser les données en ensembles d'entraînement (80%) et de test (20%)
X_train, X_test, y_train, y_test = train_test_split(X_final, y, test_size=0.2, random_state=42)

# Étape 6 : Ajuster un modèle de régression logistique
model = LogisticRegression()
model.fit(X_train, y_train)

# Étape 7 : Évaluer le modèle
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy of the model: {accuracy:.2f}")
print(classification_report(y_test, y_pred))

# Étape 8 : Estimer la probabilité d'échec sur 10 ans
X_test['Predicted_Prob_Failure'] = model.predict_proba(X_test)[:, 1]
print(X_test[['Predicted_Prob_Failure']].head())

# Étape 9 (optionnelle) : Intégrer les ajustements pour l'extrapolation temporelle
time = 15  # Exemple pour 15 ans
X_test['Adjusted_Prob_Failure'] = 1 - np.power(1 - X_test['Predicted_Prob_Failure'], time / 10)
print(X_test[['Adjusted_Prob_Failure']].head())

ModuleNotFoundError: No module named 'sklearn'

Restarted Python 3.10.11